## Use the CRaftID environment described in sequence analysis section##

**Scripts required:** 

    stage_position_from_olympus.py
    make_processing_macro_olympus.py
    
    

# 1) Generate Maximum Projected Tif files from .oir files outputted from microscope#

The example provided with this code will take .oir files from the Olympus FV3000 microscope with 3 color channels (red, blue, and brightfield) and 5 z-planes. Slight modifications may be required with other setups. 

**Example command:**

    import make_processing_macro_olympus as mpo
    import glob

    data_dir = "/home/ecwheele/scratch/testing_image_processing/input_data/"
    num_files = len(glob.glob(data_dir+"*.oir"))

    save_dir = "/home/ecwheele/scratch/testing_image_processing/tif_files/"
    !mkdir $save_dir

    common_name_prefix = "181019_raft1_A01_G001_"

    file_list = mpo.get_file_list(num_files)

    macro_file_dir = "/home/ecwheele/scratch/testing_image_processing/analysis/"
    !mkdir $macro_file_dir

    mpo.make_macro_3_channels(data_dir, save_dir, common_name_prefix, file_list, macro_file_dir)
    
**Inputs:**

    1) data_dir: Directory where .oir files exist
    
    2) tif_dir: Directory where .tif files will be saved after running the macro in Fiji
    
    3) file_name_prefix: common prefix in name of all .oir files from the experiment
    
    4) file_list: output from mpo.get_file_list(num_files)
    
    5) macro_file_dir: Folder where macro will be saved
    
**Outputs:**

    Will generate a file called oir_macro.ijm in the assigned folder that can be run in Fiji to process all oir images into a max-projection image for each color channel saved as .tif files in the assigned save_directory.
    
    Must run this file in Fiji with "Plugins" - "Macros" - "Run" to process the images.

## 2) Get absolute x and y coordinates for each image from metadata.##

This will be used later on to select rafts out of the array

**Example Command:**

    import stage_position_from_olympus as spo
    
    metadata = data_dir+"matl.omp2info"
    
    macro_file_dir = "/home/ecwheele/scratch/testing_image_processing/analysis/"
    savefile = macro_file_dir+"pos_info.csv"

    results = spo.master(metadata, filename_to_save=savefile)

**Inputs:**

    1) Metadatafile from imaging session "matl.omp2info"
    
    2) Filname and path to save result.
    
**Outputs:**

    csv file with 5 columns: filename, x-step, y-step, absolute x-coordinate, absolute y-coordinate. 

## 3) Segment tif images for individual rafts and select rafts containing cells##


**Example Command:**

    import detect_squares_v2 as grid

    array_type = "custom_100_old_10x_confocal"
    tif_file_dir = "/home/ecwheele/scratch/testing_image_processing/tif_files/"

    candidate_wells_dir = "/home/ecwheele/scratch/testing_image_processing/candidate_wells/"
    !mkdir $candidate_wells_dir

    grid.process_all(tif_file_dir, array_type, candidate_wells_dir, blur=True)
    

**Inputs:**

    1) Directory where tif files are stored, one per color channel. 
        (Current naming suffixes: Brightfield = *2.tif, Red Channel = *1.tif, Blue channel = *0.tif)
        
    2) Array type - this is used to specify the length (in pixels) of squares to be detected. 
    Need to adjust with different arrays/microscope settings (Need to make a demo on how to get these numbers). 
    
    3) Directory where individual raft images will be stored.
    
    4) Blur used to improve square detection in brightfield image. Recommended to set to True
    
**Outputs:**

    All rafts detected with cells will be saved in the assigned folder. Two images per raft, one for red and one for blue channel